# Wrangle OpenStreetMap Data

Author: [Laura Uzcategui](https://github.com/laurauzcategui/)

## Abstract 

With the amount of increased data and information over Internet and the World, one of the most crucial facts is getting right data processing, meaning cleaning and wrangling that will be most likely used for decision making such as : work, research, life and so on. 

The content of the following document is based on Exploring, Cleaning and applying Data Wrangling over an area of Dublin picked from OpenStreetMap, which is a project that creates and distributes free geographic data for the world. The most important thing is the data present in this project is open and free to contribute, which makes sometimes harder to obtain an standard as contributors could introduce data in free-format and everything that comes in free format is prone to error or it could be inaccurate.

### Scenario & Dataset 

For this project I have selected to explore the city I live in: Dublin, Ireland.

Specifically the area highlighted in the picture below. 

![Dublin Map](DublinMap.png)

### Tools used
- [OpenStreetMap](https://www.openstreetmap.org): Selected the area and coordinates of the map
- [OverPass API](https://overpass-api.de/): Used for downloading the data is OSM format matching the coordinates
- Python: used for Cleaning & DataWrangling, Insert/Delete Operations and query to Sqlite3
- Sqlite3: used as Data Storage 
- Jupyter Notebooks: Used to document the project

### Source 

- [dublin_openstreet.py](./dublin_openstreet.py): python module that was used for cleaning and wrangling of the data. 
- [dublin_db.py](./dublin_db.py): python module that contains DB class definition to allow the following operation in sqlite3: 
    - creation of database
    - connection to a database
    - drop tables 
    - create tables
    - query execution 
- [queries.py](./queries.py): python module that contains all the queries to be executed 
- [dublin_queries.py](./dublin_queries.py): Main python module that contains a CLI in order to allow the user to execute all queries or pass a list of queries to execute. 

### Background Information

Before jumping into the analysis of the data, It's important to define the following concepts: 
- [Node](https://wiki.openstreetmap.org/wiki/Elements#Node): A node represents a specific point on the earth's surface defined by its latitude and longitude. Each node comprises at least an id number and a pair of coordinates.
- [Way](https://wiki.openstreetmap.org/wiki/Elements#Way): A way is an ordered list of between 2 and 2,000 nodes that define a polyline. Ways are used to represent linear features such as rivers and roads
- [Tag](https://wiki.openstreetmap.org/wiki/Elements#Tag): All types of data element (nodes, ways and relations), as well as changesets, can have tags. Tags describe the meaning of the particular element to which they are attached.
    
    A tag consists of two free format text fields; a 'key' and a 'value'. Each of these are Unicode strings of up to 255 characters. For example, highway=residential defines the way as a road whose main function is to give access to people's homes. 

# Results 

The following script is the one executed to get the queries executed after working with the data.

In [12]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import queries
import argparse
import textwrap
import os

from prettytable import from_db_cursor
from prettytable import PrettyTable
from collections import deque

from dublin_db import DB, TABLES
from dublin_openstreet import NODES_PATH ,NODE_TAGS_PATH, WAYS_PATH, WAY_NODES_PATH, WAY_TAGS_PATH
from dublin_openstreet import expected

DB_NAME = "dublin"
QUERIES = queries.queries
QUERY_ALL = "ALL"
SPECIAL_QUERY = ["rows_per_table","street_types"]

class QUERY:
    def __init__(self):
        self.DB = DB(DB_NAME)
        self.conn = self.DB.connect_to_db()

    def print_description(self):
        return textwrap.dedent('''\
         |------------------------------------------------------------------
         | Execute your queries in the specified DB :) See the options below:
         |------------------------------------------------------------------
         | - Execute queries defined over queries.py map.
         | - Update queries.py with new queries
         | - Pass the option --query_name to execute an specific query
         | - Execute multiple queries at once by passing --queries_names option
         | Future Versions:
         | - Execute all queries
         |------------------------------------------------------------------
         ''')

    def main(self):
        ''' Cli to be able to execute a list of queries or one query only'''
        parser = argparse.ArgumentParser(description=self.print_description(),
                                         prog='dublin_queries',
                                         formatter_class=argparse.RawDescriptionHelpFormatter)
        group = parser.add_mutually_exclusive_group(required=True)
        group.add_argument('--queries_names', dest='query_name', nargs='*',
                            help='Execute a list of queries separated by an space. i.e --queries_names a b c')
        group.add_argument('--query_name', dest='query_name', help='Execute a query defined in queries.py')

        args = parser.parse_args()
        self.validate_args(args)
        return args

    def validate_args(self,args):
        ''' Validate that the query or queries passed by CLI args are valid'''
        if QUERY_ALL in args.query_name:
            return
        exist = self.query_exist(args.query_name)
        if len(exist) > 0:
            print "The following queries are not registered: {}".format(', '.join(map(str, exist)))
            exit(1)

    def query_exist(self,queries_to_check):
        ''' Will check if the query is defined at queries.py file'''
        invalid_queries = []

        queries_registered = QUERIES.keys()
        for query in queries_to_check:
            if query in queries_registered:
                continue
            else:
                invalid_queries.append(query)
        return invalid_queries

    def retrieve_query(self,query_to_lookup):
        ''' Will retrieve the query statement to be executed only if enabled'''
        if QUERIES.get(query_to_lookup) and QUERIES[query_to_lookup][1]:
            return QUERIES[query_to_lookup][0]
        else:
            return None

    def execute_queries(self,queries):
        ''' Will execute each query on the "queries" list
           if the query is on SPECIAL_QUERY it will call the method defined over the script
           else it will execute the regular query'''
        if QUERY_ALL in queries:
            queries = QUERIES.keys()
        for query in queries:
            if query in SPECIAL_QUERY:
                getattr(self, query)(query)
            else:
                statement = self.retrieve_query(query)
                if statement is not None:
                    print "Executing query_name:{}\nQuery: {}".format(query,statement)
                    rows, col_names = self.DB.execute_query(statement)
                    print format_result(deque(col_names), rows)
                else:
                    print "It looks like query: {} does not exist or it's disabled".format(query)

    def rows_per_table(self, query):
        ''' Will loop through the TABLES list and execute the same query for each table'''
        for table in TABLES:
            statement = self.retrieve_query(query)
            if statement is not None:
                st = statement.format(table)
                rows, col_names = self.DB.execute_query(st)
                print "Table: {}".format(table)
                print format_result(deque(col_names), rows)
    
    def street_types(self, query):
        ''' Will loop in street types cursor and find uniqueness'''
        st_types = set()
        statement = self.retrieve_query(query)
        if statement is not None:
            rows, col_names = self.DB.execute_query(statement)
            for row in rows:
                if row[0].split()[-1] in expected:
                    st_types.add(row[0].split()[-1])
            print "Streets types\n"
            print "\n".join(st_types)

def format_result(col_names, rows):
    ''' It will format the result of the query with a table shape'''
    ptt=PrettyTable()
    ptt.padding_width = 1
    idx = 0
    while len(col_names) > 0:
        col_name = col_names.popleft()
        if len(col_names) == 0:
            # format(row[idx], ',d')
            ptt.add_column(col_name, [ "{}".format(str(row[idx])) for row in rows])
            ptt.align[col_name]="r"
        else:
            ptt.add_column(col_name,[row[idx] for row in rows])
            ptt.align[col_name]="l"
        idx += 1
    return ptt

def print_file_size():
    ''' It will print the size of each file'''
    files = ['dublin.osm', NODES_PATH ,NODE_TAGS_PATH, WAYS_PATH, WAY_NODES_PATH, WAY_TAGS_PATH]
    file_sizeMB = []
    for file in files:
        file_size = os.path.getsize(file)
        format_str = "{} {}MB\n".format(file, float(file_size >> 20))
        file_sizeMB.append(format_str)
    print textwrap.dedent('''\
|--------------------------
| Size of Files
|--------------------------
| {}---------------------------'''.format("| ".join(file_sizeMB)))



In [13]:
# Create a QUERY object 
queries = QUERY()

### Size Of Files Stats

The following method call, will display the size of each file. 

#### Input Files
- dublin.osm 

#### Output Files
- nodes.csv
- nodes_tags.csv
- ways.csv
- ways_nodes.csv
- ways_tags.csv

In [14]:
# Get size of files
print_file_size()

|--------------------------
| Size of Files
|--------------------------
| dublin.osm 75.0MB
| nodes.csv 22.0MB
| nodes_tags.csv 2.0MB
| ways.csv 3.0MB
| ways_nodes.csv 9.0MB
| ways_tags.csv 8.0MB
---------------------------


In [15]:
# Print all Queries available 
print "\n".join(QUERIES.keys())

rows_per_table
count_unique_user_by_node
ways_most_used_keys
street_types
nodes_most_used_keys
ways_vs_nodes
nodes_count_of_streets


### Execute Queries

In the following sections you will see the execution of each query and its description and analysis. 

#### Query 1: Rows per table
Description: This query will be perform in batch and it will retrieve the amount of rows for each table. 

In [16]:
# Rows per table
queries.execute_queries(['rows_per_table'])

Table: nodes
+-----------+
| row_count |
+-----------+
|    283111 |
+-----------+
Table: nodes_tags
+-----------+
| row_count |
+-----------+
|     74044 |
+-----------+
Table: ways
+-----------+
| row_count |
+-----------+
|     62846 |
+-----------+
Table: ways_tags
+-----------+
| row_count |
+-----------+
|    251886 |
+-----------+
Table: ways_nodes
+-----------+
| row_count |
+-----------+
|    408653 |
+-----------+


#### Query 2: Count of the unique users by Node
Description: The following query contains the amount of unique users that contributed to nodes table in the selected area

In [17]:
# Rows per table
queries.execute_queries(['count_unique_user_by_node'])

Executing query_name:count_unique_user_by_node
Query: select count(distinct(user)) as distinct_users from nodes;
+----------------+
| distinct_users |
+----------------+
|            636 |
+----------------+


#### Analysis: 
The amount of unique users is impressive, 636 users are contributing to the Dublin area selected. 


##### Idea for Improvement: 
- There is not enough information if this users are residents or not of Dublin. This information will be useful to the contributors and users of openstreetdata because it can increase somehow the level of trust on the data that is being added. 

#### Query 3: Nodes most used keys

Description: The idea behind the execution of this query was to get the top keys and for that I have defined a limit on the count, therefore the query will retrieve all keys where the count was above 1000. 

In [18]:
# Rows per table
queries.execute_queries(['nodes_most_used_keys'])

Executing query_name:nodes_most_used_keys
Query: select a.* from ( select count(key) as count, key from nodes_tags group by key order by count desc ) a where a.count >= 1000;
+-------+------------------+
| count |              key |
+-------+------------------+
| 6645  |             name |
| 6152  |           street |
| 4393  |             city |
| 4153  |      housenumber |
| 4133  |          amenity |
| 3019  |          highway |
| 2847  |         operator |
| 2547  |          website |
| 1849  |          natural |
| 1711  |          barrier |
| 1696  |             shop |
| 1483  |            phone |
| 1378  |              ref |
| 1345  | public_transport |
| 1338  |       wheelchair |
| 1322  |          network |
| 1233  |              bus |
| 1147  |  traffic_calming |
| 1117  |  postal_district |
| 1075  |          shelter |
| 1003  |              lit |
+-------+------------------+


#### Analysis: 

Based on this information, it looks like on of the most contributed nodes was related to: 

- Street 
- Amenity 
- HouseNumber
- Highway

Therefore the following queries will be based on this information, Specifically queries will be executed against Street, Amenity and HouseNumber.

##### Idea for Improvement:

* By looking at the tops keys, some of them seems to be related to transportation, therefore a suggestion on tagging will be: 

    - key: bus
    - type: transportation 

    - key: highway
    - type: transportation

* Also another idea could be around merging data, for example, bus and public_transport keys could be seen as one by using the idea above.

#### Query 3: Nodes count of Streets 

Description: This query will get the count of unique number of streets reported in Nodes table. 

In [19]:
# Rows per table
queries.execute_queries(['nodes_count_of_streets'])

Executing query_name:nodes_count_of_streets
Query: select count(value) as 'Number of Streets' from (select distinct(value) as value from nodes_tags where key='street');
+-------------------+
| Number of Streets |
+-------------------+
|               895 |
+-------------------+


#### Analysis: 
There is approximately 900 streets accounted in the nodes tables in the selected Dublin area. Nevertheless I consider the tagging could be improved, I could find different types of streets like the followings: 



In [20]:
queries.execute_queries(['street_types'])

Streets types

Lane
Square
Quay
Cottages
Grove
Park
Drive
Mews
Crescent
Place
Terrace
Villas
Village
Lawn
Court
View
Avenue
Gardens
Road
Hill


##### Idea for Improvement:

* All the streets types above I consider should be marked as sub type of the tag, this way the tag definition will look like this: 

    - key: street 
    - value: Tivoli Avenue
    - type: addr
    - sub-type: Avenue
    

#### Query 4: Ways vs Nodes

Description: the purpose of the query is to analise if there is consistency between the values field when the key is "street" in the nodes_tags and ways_tags table. For achieving the goal a join between 3 tables has to be executed. 

In [21]:
queries.execute_queries(['ways_vs_nodes'])

Executing query_name:ways_vs_nodes
Query: 
select nt.id as node_id, nt.value as node_value, wt.id as way_id, wt.value as way_value
  from (
    select distinct wn.id as way, wn.node_id as node
    from ways_nodes wn, nodes_tags nt, ways_tags wt
    where wn.id = wt.id and wn.node_id = nt.id
  ) a, nodes_tags nt, ways_tags wt
where wt.key='street' and
      nt.key = 'street'and
      a.way = wt.id and
      a.node = nt.id and
      wt.value != nt.value;

+------------+---------------------+-----------+--------------------------+
| node_id    | node_value          | way_id    |                way_value |
+------------+---------------------+-----------+--------------------------+
| 516031805  | Hanover Quay        | 119453161 |            Forbes Street |
| 1443560444 | Templeville Road    | 131118307 |           Wainsfort Road |
| 2211597932 | Saint Aidan's Drive | 211151340 |          Hollywood Drive |
| 2435404944 | Charlemont Court    | 235543191 |          Charlemont Mall |
| 24354049

#### Analysis: 

In the result above, you can observe how the nodes and ways presents in the tables nodes_tags and ways_tags are present as a relationship marked in ways_nodes table. 

**Possible problems and solution**:

- Charlemount Court node
    - Problem: It's repeated 3 times with different ids and same way id. 
    - Solution: consolidate the data to be the same node. 
   

### Conclusion

The project has been a really good experience and a way to experiment with a long dataset that from the very beginning requires a lot of work from understanding the purpose of the elements to study, cleaning and wrangling the data until finally getting it shaped, so that it could be analysed and explored easily through SQL. 

I would say, there is more work to be done on this project in terms of extendig the amount of queries that could help the community to analise the data from openstreetmap and make it richer and cleaner than it is now. 

In [22]:
from subprocess import call
call(['python', '-m', 'nbconvert', 'WrangleOpenStreetMapData.ipynb'])

0